In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import classification_report_imbalanced
import keras.metrics

import visualkeras

In [2]:
# Import the BITCOIN csv into the notebook
signals_df = pd.read_csv(
    Path("./Resources/custom_new.csv"),
    index_col = 'Date',
    infer_datetime_format=True, 
    parse_dates=True
)

# Review the DataFrame
signals_df.head()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,14176944,414.140015,25.930343,22.103159,0.621316,-0.520959,1.142274,...,1,1.0,100,100.0,41523.999023,58476.000977,100000.000000,NaN,NaN,41523.999023
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,18.099063,21.674862,0.463053,-0.543377,1.006430,...,0,-1.0,0,-100.0,0.000000,99868.002319,99868.002319,-0.00132,-0.00132,-41392.001343
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,16.004308,20.011238,0.146716,-0.687771,0.834487,...,0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,11.246037,15.116469,-0.057330,-0.713454,0.656124,...,0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,5748608,411.250000,12.270360,13.173568,-0.284746,-0.752696,0.467950,...,0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000


In [3]:
# Create a list of categorical variables 
#categorical_variables = list(stoch_df.dtypes[stoch_df.dtypes == "object"].index)
categorical_variables = list(signals_df[['Signal']])
# Display the categorical variables list
display(categorical_variables[0:6])

['Signal']

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
    # sparse = False, results in an array
    # sparse = True (default), results in a sparse matrix

In [5]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(signals_df[categorical_variables])
encoded_data[0:1]

array([[0., 1.]])

In [6]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
        # function gathers column names and assigns them to the new DataFrame
)

# set index of encoded_df
encoded_df.set_index(signals_df.index, inplace=True)

# Review the DataFrame
encoded_df.head()

C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Signal_0,Signal_1
Date,,
2021-04-19 15:30:00-04:00,0.0,1.0
2021-04-20 09:30:00-04:00,1.0,0.0
2021-04-20 10:30:00-04:00,1.0,0.0
2021-04-20 11:30:00-04:00,1.0,0.0
2021-04-20 12:30:00-04:00,1.0,0.0


In [7]:
#encoded_df.drop(columns=['Signal'],inplace=True)

# Review the DataFrame
#encoded_df.head()

In [8]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
side_numeric = signals_df.drop(columns=['Signal'])

# Review the DataFrame
side_numeric.head()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,MACD_Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,14176944,414.140015,25.930343,22.103159,0.621316,-0.520959,1.142274,...,1.0,1.0,100,100.0,41523.999023,58476.000977,100000.000000,NaN,NaN,41523.999023
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,18.099063,21.674862,0.463053,-0.543377,1.006430,...,0.0,-1.0,0,-100.0,0.000000,99868.002319,99868.002319,-0.00132,-0.00132,-41392.001343
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,16.004308,20.011238,0.146716,-0.687771,0.834487,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,11.246037,15.116469,-0.057330,-0.713454,0.656124,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,5748608,411.250000,12.270360,13.173568,-0.284746,-0.752696,0.467950,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000


In [9]:
signals_ohe_df = pd.concat([encoded_df,side_numeric],axis=1)

# Review the number of columns
signals_ohe_df

,Signal_0,Signal_1,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,...,MACD_Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,0.0,1.0,415.239990,414.325012,415.339996,14176944,414.140015,25.930343,22.103159,0.621316,...,1.0,1.0,100,100.0,41523.999023,58476.000977,100000.000000,NaN,NaN,41523.999023
2021-04-20 09:30:00-04:00,1.0,0.0,413.920013,413.910004,414.679993,11437142,413.660004,18.099063,21.674862,0.463053,...,0.0,-1.0,0,-100.0,0.000000,99868.002319,99868.002319,-0.00132,-0.001320,-41392.001343
2021-04-20 10:30:00-04:00,1.0,0.0,411.575012,413.920990,413.929993,17431474,411.119995,16.004308,20.011238,0.146716,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.001320,0.000000
2021-04-20 11:30:00-04:00,1.0,0.0,412.144989,411.575012,412.399994,10103321,410.619995,11.246037,15.116469,-0.057330,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.001320,0.000000
2021-04-20 12:30:00-04:00,1.0,0.0,411.290009,412.149994,412.220001,5748608,411.250000,12.270360,13.173568,-0.284746,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.001320,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-12 12:30:00-04:00,1.0,0.0,441.640015,441.750000,442.700012,7124676,441.170013,28.591089,30.039237,-1.904712,...,1.0,-1.0,0,-100.0,0.000000,107189.407349,107189.407349,-0.00014,0.071894,-44164.001465
2022-04-12 13:30:00-04:00,1.0,0.0,438.489990,441.649994,441.940002,10061518,438.420013,14.954800,25.692469,-2.125682,...,0.0,0.0,0,0.0,0.000000,107189.407349,107189.407349,0.00000,0.071894,0.000000
2022-04-12 14:30:00-04:00,1.0,0.0,437.059998,438.480011,439.290009,14347270,436.650085,8.822045,17.455978,-2.388655,...,0.0,0.0,0,0.0,0.000000,107189.407349,107189.407349,0.00000,0.071894,0.000000


In [10]:
X = signals_ohe_df.drop(columns=['Signal_0','Signal_1'])

# Review the number of columns
len(X.columns)

23

In [11]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_ohe_df[['Signal_0','Signal_1']]

# Review the number of columns
len(y.columns)

2

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
    # test_size=x
# Review the DataFrame
X_test.head()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,MACD_Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-19 10:30:00-05:00,455.630493,456.394989,456.730011,15191549,454.730011,10.702383,12.275826,-3.000169,-0.401513,-2.598656,...,1.0,-1.0,0,-100.0,0.000000,103426.507568,103426.507568,-0.000734,0.034265,-45563.049316
2021-11-01 15:30:00-04:00,460.079987,459.369995,460.230011,8267663,459.369995,74.029651,71.058528,0.946894,0.011684,0.935211,...,1.0,1.0,100,100.0,46007.998657,56482.983398,102490.982056,0.000000,0.024910,46007.998657
2022-01-05 15:30:00-05:00,468.380005,470.299988,471.190002,21372187,468.290009,2.290720,8.693256,-1.096733,-0.979166,-0.117567,...,0.0,0.0,0,0.0,0.000000,103704.031372,103704.031372,0.000000,0.037040,0.000000
2021-09-15 10:30:00-04:00,445.290009,444.410004,445.570007,9878698,444.220001,26.847198,16.588462,-1.253207,0.081520,-1.334727,...,1.0,0.0,100,0.0,44529.000854,57410.229492,101939.230347,0.000864,0.019392,0.000000
2021-10-28 12:30:00-04:00,457.049988,457.304993,457.429993,3684041,456.500000,80.863976,70.114403,0.812403,-0.111141,0.923544,...,1.0,0.0,100,0.0,45704.998779,57049.987793,102754.986572,-0.000243,0.027550,0.000000


In [13]:
# Select the start of the training period
training_begin = X.index.min() + DateOffset(hours=1)

# Display the training begin date
print(training_begin)

2021-04-19 16:30:00-04:00


In [14]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)
    # Keep training less than 50% of total DataFrame

# Display the training end date
print(training_end)

2021-07-19 15:30:00-04:00


In [15]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,MACD_Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,18.099063,21.674862,0.463053,-0.543377,1.006430,...,0.0,-1.0,0,-100.0,0.000000,99868.002319,99868.002319,-0.00132,-0.00132,-41392.001343
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,16.004308,20.011238,0.146716,-0.687771,0.834487,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,11.246037,15.116469,-0.057330,-0.713454,0.656124,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,5748608,411.250000,12.270360,13.173568,-0.284746,-0.752696,0.467950,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 13:30:00-04:00,411.535004,411.309692,412.100006,5744918,411.200012,14.220449,12.578948,-0.440132,-0.726465,0.286333,...,1.0,1.0,100,100.0,41153.500366,58714.501953,99868.002319,0.00000,-0.00132,41153.500366


In [16]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_test.head()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,MACD_Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-20 09:30:00-04:00,429.019989,425.679993,429.470001,22332446,424.829987,27.075173,14.110055,-2.727512,-0.308362,-2.419150,...,1.0,0.0,100,0.0,42901.998901,58353.985596,101255.984497,0.004026,0.01256,0.0
2021-07-20 10:30:00-04:00,429.920013,429.019989,430.559998,11728727,428.809998,47.861584,28.632651,-2.288172,0.104782,-2.392954,...,1.0,0.0,100,0.0,42992.001343,58353.985596,101345.986938,0.000889,0.01346,0.0
2021-07-20 11:30:00-04:00,431.220001,429.920013,431.440002,9536033,429.839996,66.275627,47.070795,-1.814182,0.463018,-2.277200,...,1.0,0.0,100,0.0,43122.000122,58353.985596,101475.985718,0.001283,0.01476,0.0
2021-07-20 12:30:00-04:00,430.959991,431.220001,432.079987,9371889,430.839996,72.896502,62.344571,-1.442889,0.667449,-2.110338,...,1.0,0.0,100,0.0,43095.999146,58353.985596,101449.984741,-0.000256,0.01450,0.0
2021-07-20 13:30:00-04:00,431.799988,430.959991,431.829987,7831322,430.739990,81.425578,73.532569,-1.068539,0.833439,-1.901978,...,1.0,0.0,100,0.0,43179.998779,58353.985596,101533.984375,0.000828,0.01534,0.0


In [17]:
# Scale the features DataFrames
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

## Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# MACD Neural Network
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

23

In [19]:
# Define the number of neurons in the output layer
number_output_neurons = 2
    # equal to one because we only have one target, y.
    # should be equal to the number of target columns we are trying to predict.

In [20]:
n = '\n'

# Define the number of hidden nodes for the first hidden layer and second layer
hidden_nodes_layer1 =  (number_input_features + number_output_neurons) //2 + 20
hidden_nodes_layer2 = (hidden_nodes_layer1 + number_output_neurons) //2 + 10
hidden_nodes_layer3 = (hidden_nodes_layer2 + number_output_neurons) //2 + 10
hidden_nodes_layer4 = (hidden_nodes_layer3 + number_output_neurons) //2 + 5
hidden_nodes_layer5 = (hidden_nodes_layer4 + number_output_neurons) //2 

# Review the number hidden nodes in the first and second layer
print(f'# of neurons in the first hidden layer: {hidden_nodes_layer1}{n}# of neurons in the second hidden layer: {hidden_nodes_layer2}{n}# of neurons in the third hidden layer: {hidden_nodes_layer3}'
    f'{n}# of neurons in the fourth hidden layer: {hidden_nodes_layer4}{n}# of neurons in the fifth hidden layer: {hidden_nodes_layer5}{n}')

# of neurons in the first hidden layer: 32
# of neurons in the second hidden layer: 27
# of neurons in the third hidden layer: 24
# of neurons in the fourth hidden layer: 18
# of neurons in the fifth hidden layer: 10



In [21]:
# Create the Sequential model instance
nn = Sequential()

In [22]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

In [23]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

In [24]:
# Add the third hidden layer
nn.add(Dense(units=hidden_nodes_layer3,activation='relu'))

In [25]:
# Add the fourth hidden layer
nn.add(Dense(units=hidden_nodes_layer4,activation='relu'))

In [26]:
# Add the fifth hidden layer
nn.add(Dense(units=hidden_nodes_layer5,activation='relu'))

In [27]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons,activation='sigmoid'))

In [28]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                768       
                                                                 
 dense_1 (Dense)             (None, 27)                891       
                                                                 
 dense_2 (Dense)             (None, 24)                672       
                                                                 
 dense_3 (Dense)             (None, 18)                450       
                                                                 
 dense_4 (Dense)             (None, 10)                190       
                                                                 
 dense_5 (Dense)             (None, 2)                 22        
                                                                 
Total params: 2,993
Trainable params: 2,993
Non-trainabl

In [29]:
# Compile the Sequential model

nn.compile(loss=keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])

    # metric exploration

In [30]:
X_train_scaled

array([[-1.11743593e+00, -1.11611023e+00, -1.10069028e+00, ...,
        -2.05326763e+00, -1.76914273e+00, -2.03703726e+00],
       [-1.45290005e+00, -1.11454026e+00, -1.20958707e+00, ...,
        -2.97484867e-02, -1.76914273e+00, -1.36161910e-05],
       [-1.37136202e+00, -1.44978479e+00, -1.43173633e+00, ...,
        -2.97484867e-02, -1.76914273e+00, -1.36161910e-05],
       ...,
       [ 2.18697096e-01,  1.02842246e-01,  1.66869866e-01, ...,
        -2.97484867e-02, -2.68248258e-02, -1.36161910e-05],
       [ 8.42250618e-02,  2.22892347e-01,  1.77034630e-01, ...,
        -2.97484867e-02, -2.68248258e-02, -1.36161910e-05],
       [ 4.61887950e-01,  8.71207766e-02,  3.99183890e-01, ...,
        -2.97484867e-02, -2.68248258e-02,  2.09134104e+00]])

In [31]:
y_train

,Signal_0,Signal_1
Date,,
2021-04-20 09:30:00-04:00,1.0,0.0
2021-04-20 10:30:00-04:00,1.0,0.0
2021-04-20 11:30:00-04:00,1.0,0.0
2021-04-20 12:30:00-04:00,1.0,0.0
2021-04-20 13:30:00-04:00,0.0,1.0
...,...,...
2021-07-19 11:30:00-04:00,1.0,0.0
2021-07-19 12:30:00-04:00,1.0,0.0
2021-07-19 13:30:00-04:00,1.0,0.0


In [32]:
# Fit the model using 50 epochs and the training data
nn.fit(X_train_scaled,y_train,epochs=100, verbose=3)
    # make sure to use X_train_scaled rather than X_train
    # verbose=3, reduces the graphics displayed per epoch. in turn this increases the overall speed of the epochs.

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [33]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
nn_signals_keras = nn.evaluate(X_test_scaled,y_test, verbose=3)

# Display the model loss and accuracy results
abc = print(f'KERAS: {nn_signals_keras}')

KERAS: [0.8571726679801941, 0.892307698726654]


In [34]:
import visualkeras

visualkeras.layered_view(nn, to_file='output.png').show()

In [35]:
# Loss: 0.2134855091571808, Accuracy: 0.9853846430778503
# # of neurons in the first hidden layer: 14
# of neurons in the second hidden layer: 8
# of neurons in the third hidden layer: 5
# of neurons in the fourth hidden layer: 4
# of neurons in the fifth hidden layer: 3
# relu activations functions with softmax for the output layer
# loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy, mse']
# 3 output neurons

In [36]:
signals_df.head()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,14176944,414.140015,25.930343,22.103159,0.621316,-0.520959,1.142274,...,1,1.0,100,100.0,41523.999023,58476.000977,100000.000000,NaN,NaN,41523.999023
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,18.099063,21.674862,0.463053,-0.543377,1.006430,...,0,-1.0,0,-100.0,0.000000,99868.002319,99868.002319,-0.00132,-0.00132,-41392.001343
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,16.004308,20.011238,0.146716,-0.687771,0.834487,...,0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,11.246037,15.116469,-0.057330,-0.713454,0.656124,...,0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,5748608,411.250000,12.270360,13.173568,-0.284746,-0.752696,0.467950,...,0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000


In [37]:
signals_df.dtypes

Close                           float64
Open                            float64
High                            float64
Volume                            int64
Low                             float64
STOCHk_14_3_3                   float64
STOCHd_14_3_3                   float64
MACD_12_26_9                    float64
MACDh_12_26_9                   float64
MACDs_12_26_9                   float64
k-d                             float64
hist_adj                        float64
Stoch_Signal                    float64
MACD_Signal                     float64
Signal                            int64
Entry/Exit                      float64
Position                          int64
Entry/Exit Position             float64
Portfolio Holdings              float64
Portfolio Cash                  float64
Portfolio Total                 float64
Portfolio Period Returns        float64
Portfolio Cumulative Returns    float64
value                           float64
dtype: object

In [38]:
#categorical_variables = list(signals_df[['custom_signal']])
#categorical_variables

In [39]:
# # Encode the categorcal variables using OneHotEncoder
# encoded_data = enc.fit_transform(signals_df[categorical_variables])
# encoded_data[0:5]

In [40]:
# # Create a DataFrame with the encoded variables
# encoded_df = pd.DataFrame(
#     encoded_data,
#     columns = enc.get_feature_names(categorical_variables)
#         # function gathers column names and assigns them to the new DataFrame
# )

# # set index of encoded_df
# encoded_df.set_index(signals_df.index, inplace=True)

# # Review the DataFrame
# encoded_df.head()

In [41]:
# # Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
# side_numeric = signals_df.drop(columns=['custom_signal'])
# side_numeric.head()

In [42]:
# signals_df = pd.concat([encoded_df,side_numeric],axis=1)
# signals_df.head()

In [43]:
# X = signals_df.drop(columns=['custom_signal_-2','custom_signal_0','custom_signal_2'])

# X.head()

In [44]:
# # Create the target set selecting the Signal column and assiging it to y
# y = signals_df['Stoch_Entry/Exit']

# y

In [45]:
signals_df

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,14176944,414.140015,25.930343,22.103159,0.621316,-0.520959,1.142274,...,1,1.0,100,100.0,41523.999023,58476.000977,100000.000000,NaN,NaN,41523.999023
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,18.099063,21.674862,0.463053,-0.543377,1.006430,...,0,-1.0,0,-100.0,0.000000,99868.002319,99868.002319,-0.00132,-0.001320,-41392.001343
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,16.004308,20.011238,0.146716,-0.687771,0.834487,...,0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.001320,0.000000
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,11.246037,15.116469,-0.057330,-0.713454,0.656124,...,0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.001320,0.000000
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,5748608,411.250000,12.270360,13.173568,-0.284746,-0.752696,0.467950,...,0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.001320,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,28.591089,30.039237,-1.904712,-0.045560,-1.859152,...,0,-1.0,0,-100.0,0.000000,107189.407349,107189.407349,-0.00014,0.071894,-44164.001465
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,14.954800,25.692469,-2.125682,-0.213224,-1.912458,...,0,0.0,0,0.0,0.000000,107189.407349,107189.407349,0.00000,0.071894,0.000000
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,8.822045,17.455978,-2.388655,-0.380958,-2.007697,...,0,0.0,0,0.0,0.000000,107189.407349,107189.407349,0.00000,0.071894,0.000000


In [46]:
X=signals_df.drop(columns='Signal')
y=signals_df[['Signal']]

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
    # test_size=x
len(X_test)

436

In [48]:
y_train.value_counts()

Signal
0         803
1         503
dtype: int64

In [49]:
# Select the start of the training period
training_begin = X.index.min() + DateOffset(hours=1)

# Display the training begin date
print(training_begin)

2021-04-19 16:30:00-04:00


In [50]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)
    # Keep training less than 50% of total DataFrame

# Display the training end date
print(training_end)

2021-07-19 15:30:00-04:00


In [51]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,MACD_Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,18.099063,21.674862,0.463053,-0.543377,1.006430,...,0.0,-1.0,0,-100.0,0.000000,99868.002319,99868.002319,-0.00132,-0.00132,-41392.001343
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,16.004308,20.011238,0.146716,-0.687771,0.834487,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,11.246037,15.116469,-0.057330,-0.713454,0.656124,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,5748608,411.250000,12.270360,13.173568,-0.284746,-0.752696,0.467950,...,0.0,0.0,0,0.0,0.000000,99868.002319,99868.002319,0.00000,-0.00132,0.000000
2021-04-20 13:30:00-04:00,411.535004,411.309692,412.100006,5744918,411.200012,14.220449,12.578948,-0.440132,-0.726465,0.286333,...,1.0,1.0,100,100.0,41153.500366,58714.501953,99868.002319,0.00000,-0.00132,41153.500366


In [52]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
display(X_test.head())
display(X_test.tail())
    # NOT SURE IF DATEOFFSET IS NECESSARY FOR X/Y TEST

,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,MACD_Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-20 09:30:00-04:00,429.019989,425.679993,429.470001,22332446,424.829987,27.075173,14.110055,-2.727512,-0.308362,-2.419150,...,1.0,0.0,100,0.0,42901.998901,58353.985596,101255.984497,0.004026,0.01256,0.0
2021-07-20 10:30:00-04:00,429.920013,429.019989,430.559998,11728727,428.809998,47.861584,28.632651,-2.288172,0.104782,-2.392954,...,1.0,0.0,100,0.0,42992.001343,58353.985596,101345.986938,0.000889,0.01346,0.0
2021-07-20 11:30:00-04:00,431.220001,429.920013,431.440002,9536033,429.839996,66.275627,47.070795,-1.814182,0.463018,-2.277200,...,1.0,0.0,100,0.0,43122.000122,58353.985596,101475.985718,0.001283,0.01476,0.0
2021-07-20 12:30:00-04:00,430.959991,431.220001,432.079987,9371889,430.839996,72.896502,62.344571,-1.442889,0.667449,-2.110338,...,1.0,0.0,100,0.0,43095.999146,58353.985596,101449.984741,-0.000256,0.01450,0.0
2021-07-20 13:30:00-04:00,431.799988,430.959991,431.829987,7831322,430.739990,81.425578,73.532569,-1.068539,0.833439,-1.901978,...,1.0,0.0,100,0.0,43179.998779,58353.985596,101533.984375,0.000828,0.01534,0.0


,Close,Open,High,Volume,Low,STOCHk_14_3_3,STOCHd_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,MACD_Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns,value
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,28.591089,30.039237,-1.904712,-0.045560,-1.859152,...,1.0,-1.0,0,-100.0,0.000000,107189.407349,107189.407349,-0.00014,0.071894,-44164.001465
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,14.954800,25.692469,-2.125682,-0.213224,-1.912458,...,0.0,0.0,0,0.0,0.000000,107189.407349,107189.407349,0.00000,0.071894,0.000000
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,8.822045,17.455978,-2.388655,-0.380958,-2.007697,...,0.0,0.0,0,0.0,0.000000,107189.407349,107189.407349,0.00000,0.071894,0.000000
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,12496573,436.679993,7.240768,10.339204,-2.470944,-0.370598,-2.100347,...,1.0,0.0,0,0.0,0.000000,107189.407349,107189.407349,0.00000,0.071894,0.000000
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,0,438.290009,13.034849,9.699221,-2.505660,-0.324251,-2.181409,...,1.0,1.0,100,100.0,43829.000854,63360.406494,107189.407349,0.00000,0.071894,43829.000854


In [53]:
# Scale the features DataFrames
# Create a StandardScaler instance

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

## Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
# From SVM, instantiate SVC classifier model instance
signals_tree = RandomForestClassifier(random_state=1)
 
# Fit the model to the data using the training data
signals_tree.fit(X_train_scaled,y_train)
 
# Use the testing data to make the model predictions
y_signals_tree_pred = signals_tree.predict(X_test_scaled)

signals_tree_class = classification_report(y_test,y_signals_tree_pred)
signals_tree_matrix = confusion_matrix(y_test,y_signals_tree_pred)

C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [55]:
# Instantiate the random oversampler model

random_sampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = random_sampler.fit_resample(X_train,y_train)


y_resampled.value_counts()

# Do we have to create this before 

Signal
0         277
1         277
dtype: int64

In [56]:
# Stoch RandomForestClassifier Oversampled
signals_tree_os = RandomForestClassifier(random_state=1)
 
# Fit the model to the data using the training data
signals_tree_os.fit(X_resampled,y_resampled)
 
# Use the testing data to make the model predictions
y_signals_tree_pred_os = signals_tree_os.predict(X_test)

signals_tree_class_os = classification_report(y_test,y_signals_tree_pred_os)
signals_tree_matrix_os = confusion_matrix(y_test,y_signals_tree_pred_os)

C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [57]:
# # From LogisticRegression, instantiate LogisticRegression classifier model instance
signals_log = LogisticRegression(random_state=1)
 
 # Fit the model to the data using the training data
signals_log.fit(X_train_scaled,y_train)
 
# # Use the testing data to make the model predictions
y_signals_log_pred = signals_log.predict(X_test_scaled)

# # Create and save confusion matrix and classification report to a variable name
signals_log_matrix = confusion_matrix(y_test,y_signals_log_pred)
signals_log_class = classification_report(y_test,y_signals_log_pred)

C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [58]:
print(signals_log_matrix)
print(signals_log_class)

[[793   0]
 [  0 507]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       793
           1       1.00      1.00      1.00       507

    accuracy                           1.00      1300
   macro avg       1.00      1.00      1.00      1300
weighted avg       1.00      1.00      1.00      1300



In [59]:
# From LogisticRegression, instantiate LogisticRegression classifier model instance
signals_log_os = LogisticRegression(random_state=1)
 
# Fit the model to the data using the training data
signals_log_os.fit(X_resampled,y_resampled)
 
# # Use the testing data to make the model predictions
y_signals_log_pred_os = signals_log.predict(X_test)

# Create and save confusion matrix and classification report to a variable name
signals_log_matrix_os = confusion_matrix(y_test,y_signals_log_pred_os)
signals_log_class_os = classification_report(y_test,y_signals_log_pred_os)

C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision a

In [60]:
# NOW TRY TO TEST

In [61]:
print('NN')
print(f'KERAS:{nn_signals_keras}')
print('Random Forest')
print(signals_tree_class)
print(signals_tree_matrix)
print('OS -- Random Forest')
print(signals_tree_class_os)
print(signals_tree_matrix_os)
print('log Reg')
print(signals_log_matrix_os)
print(signals_log_class_os)
print('OS -- Log Reg')
print(signals_log_matrix_os)
print(signals_log_class_os)

NN
KERAS:[0.8571726679801941, 0.892307698726654]
Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       793
           1       1.00      1.00      1.00       507

    accuracy                           1.00      1300
   macro avg       1.00      1.00      1.00      1300
weighted avg       1.00      1.00      1.00      1300

[[793   0]
 [  0 507]]
OS -- Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       793
           1       1.00      1.00      1.00       507

    accuracy                           1.00      1300
   macro avg       1.00      1.00      1.00      1300
weighted avg       1.00      1.00      1.00      1300

[[793   0]
 [  0 507]]
log Reg
[[793   0]
 [507   0]]
              precision    recall  f1-score   support

           0       0.61      1.00      0.76       793
           1       0.00      0.00      0.00       507

    accuracy          